# Near Decision Boundry Sample Removal

In [1]:
import pickle
import numpy as np

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/3_KnnImputation/"
imputed_data = []

number_of_partitions = 5
for i in range(1,number_of_partitions +1):
# Load the array with Pickle
    with open(data_dir + "Partition" + str(i) + "_KnnImputation" + ".pkl", 'rb') as f:
        imputed_data.append(pickle.load(f))

In [2]:
import pandas as pd
labels = []
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/2_Labels/"

for i in range(1,6):
    labels.append(pd.read_csv(data_dir + "Partition" + str(i) + "_labels.csv"))

In [ ]:
# NewFeatures with Shuffle
from tqdm import tqdm

def dropping(start_partition, end_partition, data_dir, data, labels, classes_to_filter, category_mapping, name):


    num_attributes = 25
    num_timestamps = 60       

    for i in range(start_partition-1,end_partition):
        
        filtered_y_train_df = labels[i][~labels[i]['FLARE_CLASS'].isin(classes_to_filter)]
        filtered_indices = filtered_y_train_df.index
        x_train_filtered = data[i][:, :, filtered_indices]
        
        num_samples = np.array(x_train_filtered).shape[2]
        X_train = np.zeros((num_samples, num_timestamps*(num_attributes-1)))
        Y_train = np.zeros(num_samples)
        
        the_labels = pd.DataFrame()
        the_labels['FLARE_CLASS'] = filtered_y_train_df['FLARE_CLASS'].map(category_mapping)
        each_partition = np.zeros((num_timestamps, num_attributes, num_samples))
        each_partition = np.array(x_train_filtered)

        with tqdm(num_samples) as pbar:
            for j in range(0, num_samples):
                new_column = np.zeros((num_timestamps,num_attributes)) 
                new_column = each_partition[:,:,j]
                
                flettened = np.zeros(num_timestamps*(num_attributes-1))

                for m in range(1,num_attributes):
                    flettened[(m-1)*num_timestamps:m*num_timestamps] = new_column[:,m]

                X_train[j, :] = flettened
                Y_train[j] = the_labels['FLARE_CLASS'].iloc[j]
                
                pbar.update(1)

        print("P"+str(i+1)+ " "+str(X_train.shape) +" Nan-Value: "+ str(np.isnan(X_train).any()))     
        
        num_samples = X_train.shape[0]
        shuffle_indices = np.random.permutation(num_samples)

        X_train_shuffled = X_train[shuffle_indices]
        Y_train_shuffled = Y_train[shuffle_indices]
    

        with open(data_dir + "Partition" + str(i+1) 
                       + name + "_Concatenation_KnnImputation" + ".pkl", 'wb') as f:
            pickle.dump(X_train_shuffled, f)

        with open(data_dir + "Partition" + str(i+1) 
                       + "_Labels" + name "_Concatenation_KnnImputation" + ".pkl", 'wb') as f:
            pickle.dump(Y_train_shuffled, f)

## Removing B and C classes 

In [ ]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/4_5_FinalData_WithoutC_Concatenation_KnnImputation/"

classes_to_filter = ['C', 'B']
category_mapping = {'X': 1, 'M': 1,'FQ': 0}

name = "_WithoutB&C"
    
dropping(1,5,data_dir,imputed_data,labels, classes_to_filter, category_mapping, name)

## Removing C class

In [ ]:
data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/4_5_FinalData_WithoutC_Concatenation_KnnImputation/"

classes_to_filter = ['C']
category_mapping = {'X': 1, 'M': 1, 'B': 0, 'FQ': 0}

name = "_WithoutC"

dropping(1,5,data_dir,imputed_data,labels, classes_to_filter, category_mapping, name)

## Removing some of the C and M classes samples (C 9.9 to C 3.0 and M 1.6 to M 1.0)

## Concatenation